To activate the openIE jar, in the directory where OpenIE standalone is located, you have to run:


java -Xmx50g -XX:+UseConcMarkSweepGC -jar openie-assembly-5.0-SNAPSHOT.jar --httpPort 8008

In [13]:
# This is the contents of REL_HELPERS.py

from functools import partial
import pandas as pd
import sys
import warnings
from tqdm import tqdm
import re
import threading
from func_timeout import func_timeout, FunctionTimedOut
import traceback

from pyopenie import OpenIE5
#extractor = OpenIE5('http://localhost:9000')
extractors = [OpenIE5('https://10.128.0.2:8008')]#, OpenIE5('http://localhost:9000')]

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import ctypes


total_arash = []
total_rows = []



def HELPER_to_extract(sent, row, extractor):# = extractors):# None):
    # global total_arash
    # global total_rows
    total_rows = []
    arash = [] # naming one var arash so the legacy lives on...
    try:
        results = extractor.extract(sent)
        for res in results:
            for arg2 in res["extraction"]["arg2s"]:
                arash.append([res["extraction"]["arg1"]["text"], \
                    res["extraction"]["rel"]["text"], \
                        arg2["text"], res["confidence"], res["extraction"]["context"], \
                            res["extraction"]["negated"], res["extraction"]["passive"]])
    except Exception as e:
        print(e)
        return []

    # #with lock:
    #     total_arash.extend(arash)
    for i in range(len(arash)):
        total_rows.append(row)
    return arash, total_rows




def HELPER_clean_sentence(text):
    # TO DO: Refactor with other punctuations...
    text = text.replace('”','"').replace('’','\'').replace(',',',').replace('“','"')
    text1 = re.sub(r'[^\x00-\x7f]',r'', text) 
    return text1.replace("\n"," ")


def HELPER_rels_quick_clean(text):
    # expand appostrophes
    return text#.replace("'s","is").replace("'m","am").replace("'re","are")

# Method to implement extractions(?)
def OPTION1(df = None):
    # global total_arash
    # global total_rows

    df_new = pd.DataFrame()
    headers = ["postnum", "sentencenum", "relnum", "arg1", "rel", "arg2", "confidence", "context", "negated", "passive"]
    partial_sents = []
    partial_rows = []

    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
            
            # This does string preprocessing so it's not gigafucked from trying to adapt pdfs to text.
            sent = str(row["sentence:String"])
            sent1 = sent.replace('”','"').replace('’','\'').replace(',',',').replace('“','"').replace("\n","")
            sent1 = re.sub(r'[^\x00-\x7f]',r'', sent1) 
            if len(sent1) > 300 or "This file" in sent1 or "copyright law" in sent1 or "contributed equally" in sent1 or sent1.count('\n')>1:
                continue

            partial_sents.append(sent1)
            partial_rows.append(row)


            if len(partial_sents) < 1:
                continue
            else:
                # global lock
                # lock = threading.Lock()
                # threads = [threading.Thread(target=HELPER_to_extract, args=(sent, row, extractor)) for (sent,row,extractor) in zip(partial_sents, partial_rows, extractors)]
                # for t in threads: 
                #     t.start()
                # for t in threads:
                #     t.join()
                try:

                    # Here is where the money is made. This is the function that does the relationship extraction. 
                    total_arash, total_rows = HELPER_to_extract(partial_sents[0], partial_rows[0], extractors[0])
                    partial_sents = []
                    partial_rows = []
                except Exception as e:
                    print(e)
                    print("Extractor exception")
                    break#continue

            relationships = total_arash
            total_arash = []

            for k, (rel,r) in enumerate(zip(relationships, total_rows)):
                

                # This step has some ad-hocs to deal with exceptions we know of.
                # TODO: REFACTOR in simpler terms.
                dictionary = {}
                if rel[0] in ["Tcell","T-cell","T-cells","Tcells","t cell","t-cell","T cell", "T cells"]:
                    rel[0] = "T-cell"
                if rel[2] in ["Tcell","T-cell","T-cells","Tcells","t cell","t-cell","T cell", "T cells"]:
                    rel[2] = "T-cell"


                # Constructing the relationships into a format that will eventually be put into a dataframe for neptune upload
                dictionary["arg1:String"] = rel[0]
                dictionary["rel:String"] = HELPER_rels_quick_clean(rel[1])
                dictionary["arg2:String"] = rel[2]
                #dictionary["context"] = sent
                dictionary["confidence:String"] = rel[3]
                dictionary["context_rel:String"] = rel[4]
                dictionary["negated:String"] = rel[5]
                dictionary["passive:String"] = rel[6]

                for k,v in r.items():
                    if k not in ["text","postnum","sentencenum"]:
                        dictionary[k] = v
                try:        
                    df_new = df_new.append(dictionary, ignore_index=True)
                except Exception as e:
                    print(e)
                    continue

            total_rows = []

    #df_new = df_new.reindex(headers, axis=1)
    return df_new



def kill_bill_and_get_extractions(df):
    try:
        df_new = OPTION1(df)
    except Exception as e:
        df_new = pd.DataFrame()
        #tb_str = traceback.format_exc()
        #print(tb_str)
        print("Kill bill exception: ")
        print(e)
    return df_new


In [3]:
# Imports
import os
import sys
sys.path.append('/home/derek/auto-nlp/auto-nlp-bucket/NLP_dump/')

from PyPDF2 import PdfReader
import PyPDF2
import requests
from bs4 import BeautifulSoup
from scholarly import scholarly
from scholarly import ProxyGenerator
import pandas as pd
import chunk
import boto3
import nltk
from collections import defaultdict
import numpy
from numpy import real
#from REL_HELPERS import *
from io import StringIO # python3; python2: BytesIO 
import boto3
import json
import gzip
import multiprocessing
import nltk
from nltk.corpus import stopwords
import random
import string
import spacy
from celery import Celery
from celery.utils.log import get_task_logger
import traceback

/tmp/ipykernel_120875/1798218505.py:13: DeprecationWarning: 'chunk' is deprecated and slated for removal in Python 3.13
  import chunk


In [14]:
nlp = spacy.load("en_core_sci_lg")
nlp2 = spacy.load("en_core_web_sm")

def approve(i):
    words = i.split()
    for word in words:
        if word in ["Vol.","Journal","2019","2018","2020"]:
            return False
        if "www" in word:
            return False
        if ".com" in word:
            return False
        if ".edu" in word:
            return False
        if "License" in word:
            return False
        if "doi" in word or "ncbi" in word or "Vol." in word:
            return False
    if len(i) < 25:
        return False
    if i.count(",") > 6:
        return False
    if i.count(".") > 2:
        return False
    return True


# This method takes text and then returns it as a token of sentences.
# From the docs:
# 
# nltk.tokenize.sent_tokenize(text, language='english')
#
# Return a sentence-tokenized copy of text, using NLTK’s recommended 
# sentence tokenizer (currently PunktSentenceTokenizer for the 
# specified language).
#
#
# content = a string containing the text to be parsed
# from_path = a data field specific to pubmed publications that has a pointer to where the plaintext 
#             of that article is
def retrieve_metadata(content, from_path = None):

    # Get the sentences
    sent_text = nltk.sent_tokenize(content)

    buffer = ""
    text = defaultdict(list)

    for count, i in enumerate(sent_text):
        if approve(i):
            if ":" in i:
                sample = i.partition(":")[0]
                if len(sample) < 20:
                    buffer = sample
                    i = i.replace(buffer + ":", "")

            temp_context = []
            if count > 0:
                temp_context.append(sent_text[count-1])
            temp_context.append(i)
            if count < len(sent_text)-1:
                temp_context.append(sent_text[count+1])
            
            text["context:String"].append(" ".join(temp_context))
            text["sentence:String"].append(i)
            text["tag:String"].append(buffer)
            text["path:String"].append(from_path)  
    
    df = pd.DataFrame().from_dict(text)
    return df

def find_key_words(dictionary):
    doc = nlp(dictionary["sentence:String"])
    entities = doc.ents
    real_arg1 = dictionary["arg1:String"].lower()
    real_arg2 = dictionary["arg2:String"].lower()
    FLAG_first = False
    FLAG_second = False
    for ent in entities:
        if ent.text in dictionary["arg1:String"]: # overwrites because last search words are better
            real_arg1 = ent.text
            FLAG_first = True
        if ent.text in dictionary["arg2:String"]:
            real_arg2 = ent.text
            FLAG_second = True
    doc = nlp2(dictionary["sentence:String"])
    for chunk in doc.noun_chunks:
        if FLAG_first == False:
            if chunk.root.text in dictionary["arg1:String"]:
                real_arg1 = nlp2(chunk.root.text)[0].lemma_
        
        if FLAG_second == False:
            if chunk.root.text in dictionary["arg2:String"]:
                real_arg2 = nlp2(chunk.root.text)[0].lemma_
    try:
        if real_arg1[0] == dictionary["sentence:String"][0] and str.islower(real_arg1[1]):
            real_arg1 = real_arg1.lower()
    except:
        pass
    if real_arg1 == real_arg2 \
        or real_arg1.lower() in stopwords.words('english') \
        or real_arg2.lower() in stopwords.words('english'):
        return False, None, None
    else:
        return True, real_arg1, real_arg2

def return_random_string(N=20):
    return ''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(N))


# This formats data into data frames necessary for functioning
def URANUS_redone(results):
    df_nodes = {"~id":[],"name:String":[]}
    df_edges = pd.DataFrame()

    for i,row in results.iterrows():
        FLAG, real_arg1, real_arg2 = find_key_words(row)
        if FLAG:

            df_nodes["~id"].append("LIT=" + str(real_arg1))
            df_nodes["name:String"].append(str(real_arg1))

            df_nodes["~id"].append("LIT=" + str(real_arg2))
            df_nodes["name:String"].append(str(real_arg2))

            temp_row = row
            temp_row["~from"] = "LIT=" + str(real_arg1)
            temp_row["~to"] = "LIT=" + str(real_arg2)
            temp_row["~id"] = return_random_string() + "==" + "LIT=" + str(real_arg1) + "_TO_" + "LIT=" + str(real_arg2)

            df_edges = df_edges.append(temp_row, ignore_index=True)
    
    nodes_all = pd.DataFrame().from_dict(df_nodes)
    return nodes_all, df_edges


In [17]:
# We now iterate over the pubmed articles

total_nodes = pd.DataFrame()
total_edges = pd.DataFrame()

input_dir = '.'
output_dir = '.'
filename = 'wealth_of_nations_excerpt.txt'


try:
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'r') as file:
        data_lines = ''.join([line.strip() for line in file.readlines()])

    # print(dois.loc[filename[:-4]]['DOI'])
    # df = retrieve_metadata(str(data_lines), dois.loc[filename[:-4]]['DOI'])#GET DOI FOR THE PAPER
    df = retrieve_metadata(str(data_lines)) 
    print("checkpoint alpha")
    
    
    # Here is the step for calling the extraction functions
    results = kill_bill_and_get_extractions(df)
    print("checkpoint bravo")
    
    
    # This is to format the results
    temp_node_csv, temp_edge_csv = URANUS_redone(results)
    temp_node_csv["~label"] = "Scientific Literature"
    temp_edge_csv["~label"] = "Scientific Literature"
    
    
    print("Checkpoint charlie")


    total_nodes = pd.concat([total_nodes, temp_node_csv], axis=0, ignore_index=True)
    total_edges = pd.concat([total_edges, temp_edge_csv], axis=0, ignore_index=True)
    print(total_nodes.shape)
    print(total_edges.shape)
    #data_lines.close()
    

    if total_edges.shape[0] > 0: #remove after testing
        # specify the file path and file name
        #file_path_nodes = output_dir+'/df_nodes_{filename}.csv'
        #file_path_edges = output_dir+'/df_edges_{filename}.csv'

        # save the DataFrame to a CSV file
        #total_nodes.to_csv(file_path_nodes, index=False)
        #total_edges.to_csv(file_path_edges, index=False)


        # Save processed data to file in output_dir
        output_filename = f"{filename[:-4]}_processed.csv" # strip ".txt" and add "_processed.csv"
        output_filepath_nodes = os.path.join(output_dir, f"df_nodes_{output_filename}")
        output_filepath_edges = os.path.join(output_dir, f"df_edges_{output_filename}")
        total_nodes.to_csv(output_filepath_nodes, index=False)
        total_edges.to_csv(output_filepath_edges, index=False)
except Exception as e:
    print(f"Error processing {filename}: {e}")
    tb_str = traceback.format_exc()
    print(tb_str)


checkpoint alpha


  0%|          | 0/93 [00:33<?, ?it/s]


KeyboardInterrupt: 

In [23]:
import requests

k = requests.request("POST", "https://10.128.0.2:8008/getExtraction", data = "Let us go into the brave new world")

KeyboardInterrupt: 